In [ ]:
import keras, os, datetime, numpy as np, matplotlib.pyplot as plt, data, model, cv2, pickle
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, TensorBoard
from keras.optimizers import Adam, SGD

In [ ]:
batch_size = 32
training_size = 2913
testing_size = 210

train_gen = data.data_gen('dataset/images_train', 'dataset/annotations_train', batch_size=batch_size, one_hot=True)
test_gen = data.data_gen('dataset/images_test', 'dataset/annotations_test', batch_size=batch_size, one_hot=True)

In [ ]:
m = model.get_model()

In [ ]:
#m = model.get_model_without_concat()

In [ ]:
""" Weights computation for a proportional class weight distribution
y = np.zeros(1)
for i in range(20):
  _, x = next(test_gen)
  y = np.concatenate((y, np.argmax(x, axis=3).reshape(-1)))
_, weights = np.unique(y, return_counts=True)
weights = y.size / weights
"""

In [ ]:
now = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
tb = TensorBoard(log_dir=os.path.join("logs", now))
chkpt = ModelCheckpoint(filepath='backups/model.' + now + '.h5', period=5)

m.compile(optimizer=SGD(learning_rate=0.0001, momentum=.9),
          loss=model.weighted_categorical_crossentropy(weights=[0.1 if i==0 else 1 for i in range(21)]),
          metrics=['accuracy', keras.metrics.Precision()])

results = m.fit_generator(train_gen, epochs=582, steps_per_epoch=5, verbose=1, callbacks=[chkpt, tb])

In [ ]:
with open('history.{}.dump'.format(now), 'wb') as file:
    pickle.dump(results.history, file)

In [ ]:
x, y = next(test_gen)
y_hat = m.predict(x)

In [ ]:
#m.load_weights('/content/backups/model.2020_06_09-09_55_28.h5')

In [ ]:
for i in range(20):
    plt.figure(figsize=(16, 4))
    data.display_result(x, y, y_hat, i)
    plt.show()